In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:

import sys
if "ipykernel" in sys.argv[0]:
    sys.argv = [sys.argv[0]]

# Safe fallback if argparse fails in notebook
try:
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='datasets')
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--learning_rate', type=float, default=2e-5)
    parser.add_argument('--no_cache', action='store_true')
    args = parser.parse_args()
except:
    from types import SimpleNamespace
    args = SimpleNamespace(data_dir='datasets', epochs=10, batch_size=16, learning_rate=2e-5, no_cache=False)

# GPU setup for EasyOCR
import torch
import easyocr

gpu_available = torch.cuda.is_available()
reader = easyocr.Reader(['en'], gpu=gpu_available)
print("EasyOCR initialized with GPU:", gpu_available)

# LayoutLMv3 import only
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification

# Load model & processor
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=2)
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")

# Send model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/home/hasan/anaconda3/envs/gpu_ml/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/hasan/anaconda3/envs/gpu_ml/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/hasan/anaconda3/envs/gpu_ml/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/hasan/anaconda3/envs/gpu_ml/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


EasyOCR initialized with GPU: True


2025-06-13 19:30:05.533324: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-13 19:30:06.061084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-13 19:30:06.061416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-13 19:30:06.144725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 19:30:06.316233: I tensorflow/core/platform/cpu_feature_guar

LayoutLMv3ForTokenClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Encoder

In [3]:
import torch, gc

def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("✅ CUDA cache cleared.")
    gc.collect()
    print("🧹 Python garbage collection completed.")

clear_cuda_cache()


✅ CUDA cache cleared.
🧹 Python garbage collection completed.


In [4]:

import os
import torch
import numpy as np
from PIL import Image
from torch import nn
from torch.utils.data import Dataset as TorchDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import easyocr
import gc
import pickle
from tqdm import tqdm
import argparse
from datetime import datetime
import json
import tempfile
import warnings
warnings.filterwarnings("ignore")

try:
    import transformers
    print(f"Transformers version: {transformers.__version__}")
    
    try:
        from transformers import LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
        MODEL_NAME = "microsoft/layoutlmv3-base"
        PROCESSOR_CLASS = LayoutLMv3Processor
        MODEL_CLASS = LayoutLMv3ForSequenceClassification
        print("✅ Using LayoutLMv3")
    except ImportError:
        print("❌ LayoutLMv3 not available, trying LayoutLMv2...")
        try:
            from transformers import LayoutLMv2Processor, LayoutLMv2ForSequenceClassification
            MODEL_NAME = "microsoft/layoutlmv2-base-uncased"
            PROCESSOR_CLASS = LayoutLMv2Processor
            MODEL_CLASS = LayoutLMv2ForSequenceClassification
            print("✅ Using LayoutLMv2")
        except ImportError:
            print("❌ Neither LayoutLMv3 nor LayoutLMv2 available")
            print("📝 Please upgrade transformers: pip install transformers>=4.21.0")
            raise ImportError("LayoutLM models not available. Please upgrade transformers library.")
            
except ImportError as e:
    print(f"❌ Transformers library not found: {e}")
    print("📝 Please install transformers: pip install transformers>=4.21.0")
    raise

DATA_DIR = "/home/hasan/datasets/classify/test"
MAX_LENGTH = 512
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 2e-5
USE_GPU = torch.cuda.is_available()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CACHE_DIR = "./cache"
PROCESS_BATCH_SIZE = 10
SAVE_DIR = "./models"

# Enhanced CUDA environment setup
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
# Additional environment variables for CUDA stability
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use only first GPU if multiple available
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"🔧 Configuration:")
print(f"   Model: {MODEL_NAME}")
print(f"   Device: {DEVICE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Max length: {MAX_LENGTH}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Epochs: {EPOCHS}")

try:
    processor = PROCESSOR_CLASS.from_pretrained(MODEL_NAME, apply_ocr=False)
    print("✅ Processor loaded successfully")
except Exception as e:
    print(f"❌ Failed to load processor: {e}")
    raise

# Enhanced EasyOCR initialization with better error handling
def initialize_easyocr(max_retries=3):
    """Initialize EasyOCR with multiple fallback strategies"""
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempting to initialize EasyOCR (attempt {attempt + 1}/{max_retries})")
            
            # Clear GPU cache before initializing
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                torch.cuda.synchronize()
            
            # Try different initialization strategies
            if attempt == 0:
                # First attempt: Use GPU if available
                reader = easyocr.Reader(['en'], gpu=True, model_storage_directory='./easyocr_models')
            elif attempt == 1:
                # Second attempt: Force CPU mode
                print("🔄 Retrying with CPU mode...")
                reader = easyocr.Reader(['en'], gpu=False, model_storage_directory='./easyocr_models')
            else:
                # Third attempt: CPU with verbose output
                print("🔄 Final attempt with CPU and verbose output...")
                reader = easyocr.Reader(['en'], gpu=False, verbose=True, model_storage_directory='./easyocr_models')
            
            # Test the reader with a simple operation
            test_image = np.ones((100, 100, 3), dtype=np.uint8) * 255
            _ = reader.readtext(test_image, detail=0)
            
            print("✅ EasyOCR initialized successfully")
            return reader
            
        except Exception as e:
            print(f"❌ EasyOCR initialization attempt {attempt + 1} failed: {e}")
            
            if "CUDA" in str(e) or "device-side assert" in str(e):
                print("🔧 CUDA error detected, will try CPU mode...")
                USE_GPU = False
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    torch.cuda.synchronize()
            
            if attempt == max_retries - 1:
                print("❌ All EasyOCR initialization attempts failed")
                raise e
    
    return None

# Initialize EasyOCR with enhanced error handling
try:
    reader = initialize_easyocr()
except Exception as e:
    print(f"❌ Failed to initialize EasyOCR after all attempts: {e}")
    print("📝 Possible solutions:")
    print("   1. Restart your Python session")
    print("   2. Clear GPU memory: nvidia-smi --gpu-reset")
    print("   3. Install EasyOCR: pip install easyocr")
    print("   4. Try running on CPU only")
    raise

def clear_memory():
    """Enhanced memory clearing function"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()

def get_memory_usage():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**3
    return 0

def load_data(data_dir):
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory not found: {data_dir}")
    
    samples = []
    label_map = {}
    label_id = 0

    for label_name in sorted(os.listdir(data_dir)):
        label_path = os.path.join(data_dir, label_name)
        if not os.path.isdir(label_path):
            continue

        if label_name not in label_map:
            label_map[label_name] = label_id
            label_id += 1

        for file in os.listdir(label_path):
            if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                samples.append({
                    "image_path": os.path.join(label_path, file),
                    "label": label_map[label_name],
                    "filename": file
                })

    print(f"📊 Dataset Summary:")
    print(f"   Unique labels: {sorted(label_map.items(), key=lambda x: x[1])}")
    print(f"   Total samples: {len(samples)}")
    
    if len(samples) == 0:
        raise ValueError("No image files found in the dataset directory")
    
    return samples, label_map

def safe_save_cache(data, filepath):
    temp_file = None
    try:
        temp_dir = os.path.dirname(filepath)
        with tempfile.NamedTemporaryFile(dir=temp_dir, delete=False, suffix='.tmp') as f:
            temp_file = f.name
            pickle.dump(data, f)
        
        os.rename(temp_file, filepath)
        return True
    except Exception as e:
        print(f"Failed to save cache {filepath}: {e}")
        if temp_file and os.path.exists(temp_file):
            os.unlink(temp_file)
        return False

def safe_load_cache(filepath):
    if not os.path.exists(filepath):
        return None
    
    try:
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        return data
    except (EOFError, pickle.UnpicklingError, Exception) as e:
        print(f"Cache file corrupted: {filepath}, error: {e}")
        print("Deleting corrupted cache file...")
        try:
            os.unlink(filepath)
        except:
            pass
        return None

def validate_and_fix_boxes(boxes, image_width, image_height):
    fixed_boxes = []
    for box in boxes:
        x0, y0, x1, y1 = box
        
        x0 = max(0, min(x0, image_width - 1))
        y0 = max(0, min(y0, image_height - 1))
        x1 = max(x0 + 1, min(x1, image_width))
        y1 = max(y0 + 1, min(y1, image_height))
        
        if x1 - x0 < 1:
            x1 = x0 + 1
        if y1 - y0 < 1:
            y1 = y0 + 1
            
        fixed_boxes.append([int(x0), int(y0), int(x1), int(y1)])
    
    return fixed_boxes

def safe_ocr_processing(image_array, max_retries=2):
    """Safely perform OCR with error handling"""
    for attempt in range(max_retries + 1):
        try:
            # Clear memory before OCR
            clear_memory()
            
            # Ensure image is in correct format
            if len(image_array.shape) == 3 and image_array.shape[2] == 3:
                # Convert to uint8 if needed
                if image_array.dtype != np.uint8:
                    image_array = image_array.astype(np.uint8)
                
                results = reader.readtext(image_array, detail=1, paragraph=False)
                return results
            else:
                print(f"Invalid image format: {image_array.shape}")
                return []
                
        except Exception as ocr_error:
            print(f"OCR attempt {attempt + 1} failed: {ocr_error}")
            clear_memory()
            
            if attempt == max_retries:
                print("All OCR attempts failed, returning empty results")
                return []
    
    return []

def process_single(sample, max_retries=2):
    for attempt in range(max_retries + 1):
        try:
            image_path = sample["image_path"]
            
            # Load and preprocess image
            image = Image.open(image_path).convert("RGB")
            original_width, original_height = image.size
            
            # Resize if too large
            max_size = 1024
            if max(image.size) > max_size:
                ratio = max_size / max(image.size)
                new_size = tuple(int(dim * ratio) for dim in image.size)
                image = image.resize(new_size, Image.Resampling.LANCZOS)
            
            current_width, current_height = image.size
            img_np = np.array(image)
            
            # Perform OCR with enhanced error handling
            results = safe_ocr_processing(img_np)
            
            words, boxes = [], []
            
            for detection in results:
                try:
                    box, text, conf = detection
                    if conf > 0.3 and text.strip():
                        clean_text = text.strip().replace('\n', ' ').replace('\t', ' ')
                        if len(clean_text) > 0:
                            words.append(clean_text)
                            
                            x_coords = [pt[0] for pt in box]
                            y_coords = [pt[1] for pt in box]
                            x0 = min(x_coords)
                            y0 = min(y_coords)
                            x1 = max(x_coords)
                            y1 = max(y_coords)
                            
                            boxes.append([x0, y0, x1, y1])
                except Exception as e:
                    print(f"Error processing OCR result: {e}")
                    continue

            # Handle empty OCR results
            if not words:
                words = ["[NO_TEXT_DETECTED]"]
                boxes = [[0, 0, min(50, current_width-1), min(20, current_height-1)]]

            boxes = validate_and_fix_boxes(boxes, current_width, current_height)
            
            # Limit number of words to prevent memory issues
            max_words = min(len(words), MAX_LENGTH // 4)
            if len(words) > max_words:
                words = words[:max_words]
                boxes = boxes[:max_words]

            if not words:
                words = ["[EMPTY]"]
                boxes = [[0, 0, min(50, current_width-1), min(20, current_height-1)]]

            print(f"Processing {sample['filename']}: {len(words)} words detected")

            # Process with transformers
            try:
                encoded = processor(
                    images=image,
                    text=words,
                    boxes=boxes,
                    truncation=True,
                    padding="max_length",
                    max_length=MAX_LENGTH,
                    return_tensors="pt",
                    return_attention_mask=True,
                    return_token_type_ids=True if 'v2' in MODEL_NAME.lower() else False
                )
            except Exception as proc_error:
                print(f"Processor error for {sample['filename']}: {proc_error}")
                # Fallback to minimal processing
                encoded = processor(
                    images=Image.new("RGB", (224, 224), "white"),
                    text=["[ERROR]"],
                    boxes=[[0, 0, 50, 20]],
                    truncation=True,
                    padding="max_length",
                    max_length=MAX_LENGTH,
                    return_tensors="pt"
                )

            # Prepare result
            result = {}
            for k, v in encoded.items():
                if isinstance(v, torch.Tensor):
                    tensor = v.squeeze(0)
                    
                    if k == 'input_ids':
                        vocab_size = processor.tokenizer.vocab_size
                        tensor = torch.clamp(tensor, 0, vocab_size - 1)
                    
                    result[k] = tensor
                else:
                    result[k] = v

            label_value = sample["label"]
            if not isinstance(label_value, int) or label_value < 0:
                print(f"Invalid label for {sample['filename']}: {label_value}")
                label_value = 0
                
            result["labels"] = torch.tensor(label_value, dtype=torch.long)
            
            # Validate tensor shapes
            expected_length = MAX_LENGTH
            for key in ['input_ids', 'attention_mask']:
                if key in result and result[key].shape[0] != expected_length:
                    print(f"Warning: {key} has unexpected length {result[key].shape[0]}, expected {expected_length}")
            
            # Clean up
            del image, img_np, encoded
            clear_memory()
            
            return result

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {sample['filename']}: {e}")
            clear_memory()
            
            if attempt == max_retries:
                print(f"Creating minimal dummy sample for {sample['filename']}")
                try:
                    dummy_image = Image.new("RGB", (224, 224), color="white")
                    dummy = processor(
                        images=dummy_image,
                        text=["[ERROR_FALLBACK]"],
                        boxes=[[0, 0, 50, 20]],
                        truncation=True,
                        padding="max_length",  
                        max_length=MAX_LENGTH,
                        return_tensors="pt"
                    )
                    dummy = {k: v.squeeze(0) for k, v in dummy.items()}
                    dummy["labels"] = torch.tensor(sample.get("label", 0), dtype=torch.long)
                    return dummy
                except Exception as dummy_error:
                    print(f"Even dummy creation failed: {dummy_error}")
                    return None

def process_samples_in_batches(samples, batch_size=PROCESS_BATCH_SIZE):
    processed_samples = []
    
    cache_pattern = os.path.join(CACHE_DIR, "chunk_*.pkl")
    import glob
    existing_chunks = sorted(glob.glob(cache_pattern))
    
    if existing_chunks:
        print(f"📦 Found {len(existing_chunks)} existing cache chunks")
        print("🔄 Loading from cache chunks...")
        
        for chunk_file in tqdm(existing_chunks, desc="Loading cache chunks"):
            chunk_data = safe_load_cache(chunk_file)
            if chunk_data:
                valid_samples = []
                for sample in chunk_data:
                    if sample is not None and 'labels' in sample:
                        valid_samples.append(sample)
                processed_samples.extend(valid_samples)
            else:
                print(f"Skipping corrupted chunk: {os.path.basename(chunk_file)}")
        
        print(f"✅ Loaded {len(processed_samples)} valid samples from cache")
        
        if len(processed_samples) >= len(samples):
            return processed_samples[:len(samples)]
        else:
            print(f"⚠️  Cache incomplete ({len(processed_samples)}/{len(samples)}), processing remaining...")
            remaining_samples = samples[len(processed_samples):]
            start_chunk = len(existing_chunks)
    else:
        print(f"🔄 Processing {len(samples)} samples in batches of {batch_size}...")
        remaining_samples = samples
        start_chunk = 0
    
    if 'remaining_samples' in locals() and remaining_samples:
        for i in tqdm(range(0, len(remaining_samples), batch_size), desc="Processing batches"):
            batch = remaining_samples[i:i + batch_size]
            batch_processed = []
            
            for j, sample in enumerate(batch):
                try:
                    processed = process_single(sample)
                    if processed is not None:
                        batch_processed.append(processed)
                    
                    current_idx = len(processed_samples) + j + 1
                    if current_idx % 50 == 0:
                        mem_usage = get_memory_usage()
                        print(f"Processed {current_idx}/{len(samples)} samples. GPU memory: {mem_usage:.2f}GB")
                    
                except Exception as e:
                    print(f"Failed to process sample {len(processed_samples) + j}: {e}")
                    continue
            
            if batch_processed:
                chunk_num = start_chunk + (i // batch_size)
                chunk_file = os.path.join(CACHE_DIR, f"chunk_{chunk_num:04d}.pkl")
                
                if safe_save_cache(batch_processed, chunk_file):
                    print(f"💾 Saved chunk {chunk_num} with {len(batch_processed)} samples")
                else:
                    print(f"❌ Failed to save chunk {chunk_num}")
                
                processed_samples.extend(batch_processed)
            
            clear_memory()
    
    print(f"✅ Successfully processed {len(processed_samples)} samples")
    return processed_samples

class MemoryEfficientDataset(TorchDataset):
    def __init__(self, samples):
        self.samples = []
        for sample in samples:
            if sample is not None and 'labels' in sample and 'input_ids' in sample:
                self.samples.append(sample)
        
        print(f"Dataset initialized with {len(self.samples)} valid samples")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if idx >= len(self.samples):
            raise IndexError(f"Index {idx} out of range for dataset of size {len(self.samples)}")
            
        sample = self.samples[idx]
        
        result = {}
        for k, v in sample.items():
            if isinstance(v, torch.Tensor):
                result[k] = v.clone().detach()
                
                if k == 'input_ids':
                    max_token_id = processor.tokenizer.vocab_size - 1
                    result[k] = torch.clamp(result[k], 0, max_token_id)
                    
            else:
                result[k] = torch.tensor(v) if not isinstance(v, torch.Tensor) else v
                
        return result

def train_epoch(model, dataloader, optimizer, epoch):
    model.train()
    total_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(dataloader, desc=f"Training Epoch {epoch}")
    
    for batch_idx, batch in enumerate(progress_bar):
        try:
            if not batch or 'labels' not in batch:
                print(f"Skipping invalid batch {batch_idx}")
                continue
                
            try:
                for k in batch:
                    if isinstance(batch[k], torch.Tensor):
                        batch[k] = batch[k].to(DEVICE, non_blocking=True)
                        
                        if k == 'input_ids':
                            max_val = batch[k].max().item()
                            vocab_size = model.config.vocab_size
                            if max_val >= vocab_size:
                                print(f"Warning: Found token ID {max_val} >= vocab_size {vocab_size} in batch {batch_idx}")
                                batch[k] = torch.clamp(batch[k], 0, vocab_size - 1)
                                
            except Exception as move_error:
                print(f"Error moving batch {batch_idx} to device: {move_error}")
                continue
            
            optimizer.zero_grad()
            
            try:
                outputs = model(**batch)
                loss = outputs.loss
            except RuntimeError as forward_error:
                if "index" in str(forward_error).lower() or "assert" in str(forward_error).lower():
                    print(f"Index/assertion error in batch {batch_idx}: {forward_error}")
                    print(f"Batch shapes: {[(k, v.shape if hasattr(v, 'shape') else type(v)) for k, v in batch.items()]}")
                    continue
                else:
                    raise forward_error
            
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"Skipping batch {batch_idx} due to invalid loss: {loss}")
                continue
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            if batch_idx % 10 == 0:
                clear_memory()
                
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"GPU OOM in batch {batch_idx}, clearing cache...")
                clear_memory()
                continue
            elif "index" in str(e).lower() or "assert" in str(e).lower():
                print(f"Index/assertion error in batch {batch_idx}: {e}")
                continue
            else:
                print(f"Runtime error in batch {batch_idx}: {e}")
                continue
        except Exception as e:
            print(f"Unexpected error in batch {batch_idx}: {e}")
            continue
    
    if num_batches == 0:
        print("Warning: No batches processed successfully!")
        return float('inf')
        
    avg_loss = total_loss / num_batches
    print(f"Average training loss: {avg_loss:.4f}")
    return avg_loss

def evaluate_model(model, dataloader, label_map, title="Validation"):
    model.eval()
    preds, trues = [], []
    
    progress_bar = tqdm(dataloader, desc=f"Evaluating {title}")
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(progress_bar):
            try:
                if not batch or 'labels' not in batch:
                    continue
                    
                labels = batch['labels'].cpu().numpy()
                
                for k in batch:
                    if isinstance(batch[k], torch.Tensor):
                        batch[k] = batch[k].to(DEVICE, non_blocking=True)
                        
                        if k == 'input_ids':
                            max_val = batch[k].max().item()
                            if max_val >= model.config.vocab_size:
                                batch[k] = torch.clamp(batch[k], 0, model.config.vocab_size - 1)
                
                outputs = model(**batch)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1).cpu().numpy()
                
                preds.extend(predictions)
                trues.extend(labels)
                
                if batch_idx % 10 == 0:
                    clear_memory()
                    
            except Exception as e:
                print(f"Error in evaluation batch {batch_idx}: {e}")
                continue

    if len(preds) == 0:
        print(f"No valid predictions for {title}")
        return 0.0, 0.0

    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='weighted', zero_division=0)
    
    print(f"\n📊 {title} Metrics:")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   F1-Score (weighted): {f1:.4f}")
    
    print(f"\n📋 Classification Report ({title}):")
    print(classification_report(trues, preds, target_names=list(label_map.keys()), zero_division=0))

    cm = confusion_matrix(trues, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", 
                xticklabels=list(label_map.keys()), 
                yticklabels=list(label_map.keys()), 
                cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix ({title})")
    plt.tight_layout()
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f"confusion_matrix_{title.lower()}_{timestamp}.png", dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()
    
    return accuracy, f1

def split_data(samples, train_ratio=0.8, val_ratio=0.1):
    np.random.shuffle(samples)
    n = len(samples)
    
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    
    train_samples = samples[:train_end]
    val_samples = samples[train_end:val_end]
    test_samples = samples[val_end:]
    
    print(f"📊 Data Split:")
    print(f"   Training: {len(train_samples)} samples")
    print(f"   Validation: {len(val_samples)} samples")
    print(f"   Test: {len(test_samples)} samples")
    
    return train_samples, val_samples, test_samples

def save_model_and_results(model, label_map, results, timestamp):
    try:
        model_path = os.path.join(SAVE_DIR, f"layoutlm_model_{timestamp}")
        model.save_pretrained(model_path)
        
        processor.save_pretrained(model_path)
        
        metadata = {
            'label_map': label_map,
            'results': results,
            'model_name': MODEL_NAME,
            'config': {
                'max_length': MAX_LENGTH,
                'batch_size': BATCH_SIZE,
                'epochs': EPOCHS,
                'learning_rate': LEARNING_RATE
            }
        }
        
        metadata_path = os.path.join(model_path, 'metadata.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2, default=str)
        
        print(f"💾 Model and metadata saved to: {model_path}")
        
    except Exception as e:
        print(f"❌ Failed to save model: {e}")

Transformers version: 4.52.4
✅ Using LayoutLMv3
🔧 Configuration:
   Model: microsoft/layoutlmv3-base
   Device: cuda
   Batch size: 1
   Max length: 512
   Learning rate: 2e-05
   Epochs: 3
✅ Processor loaded successfully
🔄 Attempting to initialize EasyOCR (attempt 1/3)
✅ EasyOCR initialized successfully


In [ ]:
def main(data_dir=None, epochs=None, batch_size=None, learning_rate=None, no_cache=False):
    data_dir = data_dir or DATA_DIR
    epochs = epochs or EPOCHS
    batch_size = batch_size or BATCH_SIZE
    learning_rate = learning_rate or LEARNING_RATE
    
    print(f"🚀 Starting LayoutLM Document Classification Training")
    print(f"   Data directory: {data_dir}")
    print(f"   Epochs: {epochs}")
    print(f"   Batch size: {batch_size}")
    print(f"   Learning rate: {learning_rate}")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("\n📂 Loading dataset...")
    samples, label_map = load_data(data_dir)
    
    train_samples, val_samples, test_samples = split_data(samples)
    
    if no_cache:
        print("🗑️  Clearing cache as requested...")
        import glob
        cache_files = glob.glob(os.path.join(CACHE_DIR, "chunk_*.pkl"))
        for cache_file in cache_files:
            try:
                os.unlink(cache_file)
            except:
                pass
    
    print("🔄 Processing samples with enhanced validation...")
    all_processed = process_samples_in_batches(samples)
    
    if len(all_processed) == 0:
        raise ValueError("No samples were processed successfully! Check your data and configuration.")
        
    if len(all_processed) != len(samples):
        print(f"⚠️  Warning: Only {len(all_processed)}/{len(samples)} samples processed successfully")
    
    train_processed = all_processed[:len(train_samples)]
    val_processed = all_processed[len(train_samples):len(train_samples)+len(val_samples)]
    test_processed = all_processed[len(train_samples)+len(val_samples):]
    
    print("\n🔧 Creating datasets and dataloaders...")
    train_dataset = MemoryEfficientDataset(train_processed)
    val_dataset = MemoryEfficientDataset(val_processed)
    test_dataset = MemoryEfficientDataset(test_processed)
    num_work=os.cpu_count() //2
    train_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        num_workers=6,
        pin_memory=True,
        persistent_workers=True
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=2,
        shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=2,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    print(f"📊 DataLoader Summary:")
    print(f"   Training batches: {len(train_loader)}")
    print(f"   Validation batches: {len(val_loader)}")
    print(f"   Test batches: {len(test_loader)}")
    
    print("\n🧠 Initializing model...")
    num_labels = len(label_map)
    model = MODEL_CLASS.from_pretrained(MODEL_NAME, num_labels=num_labels)
    model.to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=1.0, 
        end_factor=0.1, 
        total_iters=epochs * len(train_loader)
    )
    
    print(f"🎯 Model Summary:")
    print(f"   Model class: {MODEL_CLASS.__name__}")
    print(f"   Number of labels: {num_labels}")
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    
    results = {
        'train_losses': [],
        'val_accuracies': [],
        'val_f1_scores': [],
        'best_val_accuracy': 0.0,
        'best_val_f1': 0.0
    }
    
    print(f"\n🏋️ Starting training for {epochs} epochs...")
    best_val_f1 = 0.0
    
    for epoch in range(1, epochs + 1):
        print(f"\n{'='*50}")
        print(f"EPOCH {epoch}/{epochs}")
        print(f"{'='*50}")
        
        train_loss = train_epoch(model, train_loader, optimizer, epoch)
        results['train_losses'].append(train_loss)
        
        val_accuracy, val_f1 = evaluate_model(model, val_loader, label_map, "Validation")
        results['val_accuracies'].append(val_accuracy)
        results['val_f1_scores'].append(val_f1)
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            results['best_val_accuracy'] = val_accuracy
            results['best_val_f1'] = val_f1
            
            print(f"🏆 New best validation F1: {val_f1:.4f}")
            
            best_model_path = os.path.join(SAVE_DIR, f"best_model_{timestamp}")
            try:
                model.save_pretrained(best_model_path)
                processor.save_pretrained(best_model_path)
                print(f"💾 Best model saved to: {best_model_path}")
            except Exception as e:
                print(f"❌ Failed to save best model: {e}")
        
        scheduler.step()
        clear_memory()
        
        print(f"📈 Epoch {epoch} Summary:")
        print(f"   Train Loss: {train_loss:.4f}")
        print(f"   Val Accuracy: {val_accuracy:.4f}")
        print(f"   Val F1: {val_f1:.4f}")
        print(f"   Best Val F1: {best_val_f1:.4f}")
    
    print("\n🧪 Running final test evaluation...")
    test_accuracy, test_f1 = evaluate_model(model, test_loader, label_map, "Test")
    results['test_accuracy'] = test_accuracy
    results['test_f1'] = test_f1
    
    print("\n📊 Final Training Summary:")
    print(f"   Best Validation Accuracy: {results['best_val_accuracy']:.4f}")
    print(f"   Best Validation F1: {results['best_val_f1']:.4f}")
    print(f"   Final Test Accuracy: {test_accuracy:.4f}")
    print(f"   Final Test F1: {test_f1:.4f}")
    
    # Save final model and results
    save_model_and_results(model, label_map, results, timestamp)
    
    # Plot training curves
    plt.figure(figsize=(15, 5))
    
    # Plot training loss
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs + 1), results['train_losses'], 'b-', marker='o')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # Plot validation accuracy
    plt.subplot(1, 3, 2)
    plt.plot(range(1, epochs + 1), results['val_accuracies'], 'g-', marker='o')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    
    # Plot validation F1
    plt.subplot(1, 3, 3)
    plt.plot(range(1, epochs + 1), results['val_f1_scores'], 'r-', marker='o')
    plt.title('Validation F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"training_curves_{timestamp}.png", dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()
    
    print(f"\n✅ Training completed successfully!")
    print(f"📁 All outputs saved with timestamp: {timestamp}")
    
    return {
        'model': model,
        'processor': processor,
        'label_map': label_map,
        'results': results,
        'timestamp': timestamp
    }


if __name__ == "__main__":
    try:
        main(
            data_dir="/home/hasan/datasets/classify/test",
            epochs=15,
            batch_size=32,
            learning_rate=2e-5,
            no_cache=True
        )
        print("🎉 Training pipeline completed successfully!")
        
    except KeyboardInterrupt:
        print("\n⏹️  Training interrupted by user")
    except Exception as e:
        print(f"\n❌ Training failed with error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        clear_memory()
        print("🧹 Memory cleanup completed")

🚀 Starting LayoutLM Document Classification Training
   Data directory: /home/hasan/datasets/classify/test
   Epochs: 15
   Batch size: 32
   Learning rate: 2e-05

📂 Loading dataset...
📊 Dataset Summary:
   Unique labels: [('ID', 0), ('advertisement', 1), ('budget', 2), ('email', 3), ('file_folder', 4), ('form', 5), ('handwritten', 6), ('invoice', 7), ('letter', 8), ('memo', 9), ('news_article', 10), ('presentation', 11), ('questionnaire', 12), ('receipt', 13), ('resume', 14), ('scientific_publication', 15), ('scientific_report', 16), ('specification', 17)]
   Total samples: 3244
📊 Data Split:
   Training: 2595 samples
   Validation: 324 samples
   Test: 325 samples
🗑️  Clearing cache as requested...
🔄 Processing samples with enhanced validation...
🔄 Processing 3244 samples in batches of 10...


Processing batches:   0%|                                                             | 0/325 [00:00<?, ?it/s]

Processing grc_passport_70.jpg: 4 words detected
Processing img_08214.png: 13 words detected
Processing img_07224.png: 11 words detected
Processing lva_passport_89.jpg: 15 words detected
Processing img_08337.png: 6 words detected
Processing img_07152.png: 1 words detected
Processing svk_id_61.jpg: 13 words detected
Processing img_07310.png: 5 words detected
Processing img_07838.png: 22 words detected
Processing img_06937.png: 13 words detected


Processing batches:   0%|▏                                                    | 1/325 [00:08<44:29,  8.24s/it]

💾 Saved chunk 0 with 10 samples
Processing img_07562.png: 11 words detected
Processing grc_passport_94.jpg: 17 words detected
Processing img_08766.png: 16 words detected
Processing img_08082.png: 19 words detected
Processing lva_passport_97.jpg: 37 words detected
Processing srb_passport_90.jpg: 18 words detected
Processing img_06488.png: 7 words detected
Processing img_08190.png: 8 words detected
Processing img_07158.png: 1 words detected
Processing img_08238.png: 7 words detected
💾 Saved chunk 1 with 10 samples


...

Processing batches: 100%|███████████████████████████████████████████████████| 325/325 [39:10<00:00,  7.23s/it]

💾 Saved chunk 324 with 4 samples
✅ Successfully processed 3244 samples

🔧 Creating datasets and dataloaders...
Dataset initialized with 2595 valid samples
Dataset initialized with 324 valid samples
Dataset initialized with 325 valid samples
📊 DataLoader Summary:
   Training batches: 649
   Validation batches: 162
   Test batches: 163

🧠 Initializing model...



Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🎯 Model Summary:
   Model class: LayoutLMv3ForSequenceClassification
   Number of labels: 18
   Parameters: 125,931,410
   Trainable parameters: 125,931,410

🏋️ Starting training for 15 epochs...

EPOCH 1/15


Training Epoch 1:  20%|███████▌                              | 129/649 [27:47<2:02:48, 14.17s/it, loss=0.0111]